<a href="https://colab.research.google.com/github/Oviing/Information-Retrieval/blob/main/IR_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [ ]:
#download the facebokk bart model for summarisation

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
  
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

In [ ]:
# load the debate data

trump_clinton_1 = pd.read_excel('/content/trump_clinton.xlsx')
trump_clinton_2 = pd.read_excel('/content/trump_clinton_2.xlsx')
trump_clinton_3 = pd.read_excel('/content/trump_clinton_3.xlsx')

vp_2016 = pd.read_excel('/content/pence_kaine.xlsx')
vp_2020_raw = pd.read_excel('/content/vice_president_debate.xlsx')
vp_2020 = vp_2020_raw[vp_2020_raw['Label'] != 7]

trump_townhall = pd.read_excel('/content/trump_town_hall.xlsx')
biden_townhall = pd.read_excel('/content/biden_town_hall.xlsx')

trump_biden_1 = pd.read_excel('/content/president_debate_1.xlsx')
trump_biden_2 = pd.read_excel('/content/president_debate_2.xlsx')

In [ ]:
#create one dataframe

data = pd.concat([trump_clinton_1, trump_clinton_2, trump_clinton_3
                  , vp_2016, vp_2020,
                  trump_townhall, biden_townhall,
                  trump_biden_1, trump_biden_2], axis = 0)

In [ ]:
# clean the data by dropping na values

data = data.dropna()

In [ ]:
#split the original data in a 50% test and training set

train, test = train_test_split(data, test_size=0.5, random_state=13, shuffle=True, stratify=data['Label'])

In [ ]:
train.head()

,Statement,Answer,Label
34,You were very much involved -- excuse me. My t...,But I think it's really an important issue. He...,4.0
73,I do rule out banning fracking because the ans...,Excuse me. He was against fracking. He said it...,3.0
1,"Which means that you will, in effect, determin...","Well, first of all, it's great to be with you,...",4.0
4,"But also, state that you are going to eliminat...",I carry this card with me. When I said the Tru...,6.0
52,What is radical about racial sensitivity train...,The fact is that there is racial insensitivity...,6.0


In [ ]:
test.to_csv('test_data.csv')

In [ ]:
# create the final data by augmenting text to increase the training set size. 
# We perform text augmentation by summarisation

summary_list = []
for i in range(0, len(train)):
  try:
    s = train['Statement'].iloc[i]
    tokenized_s = tokenizer(s,return_tensors='pt')

    a = train['Answer'].iloc[i]
    tokenized_a = tokenizer(a,return_tensors='pt')

    label = train['Label'].iloc[i]


    summary_s = model.generate(tokenized_s['input_ids'])
    summary_a = model.generate(tokenized_a['input_ids'])


    decoded_s = tokenizer.batch_decode(summary_s)
    decoded_a = tokenizer.batch_decode(summary_a)

    summary_list.append((s, a, label, 'raw'))
    summary_list.append((decoded_s, decoded_a, label, 'summary'))
  except IndexError:
    pass
  print((i / len(train))* 100)

0.0
0.5128205128205128
1.0256410256410255
1.5384615384615385
2.051282051282051
2.564102564102564
3.076923076923077
3.5897435897435894
4.102564102564102
4.615384615384616
5.128205128205128
5.641025641025641
6.153846153846154
6.666666666666667
7.179487179487179
7.6923076923076925
8.205128205128204
8.717948717948717
9.230769230769232
9.743589743589745
10.256410256410255
10.76923076923077
11.282051282051283
11.794871794871794
12.307692307692308
12.82051282051282
13.333333333333334
13.846153846153847
14.358974358974358
14.871794871794872
15.384615384615385
15.897435897435896
16.41025641025641
16.923076923076923
17.435897435897434
17.94871794871795
18.461538461538463
18.974358974358974
19.48717948717949
20.0
20.51282051282051
21.025641025641026
21.53846153846154
22.05128205128205
22.564102564102566
23.076923076923077
23.589743589743588
24.102564102564102
24.615384615384617
25.128205128205128
25.64102564102564
26.153846153846157
26.666666666666668
27.17948717948718
27.692307692307693
28.20512

Token indices sequence length is longer than the specified maximum sequence length for this model (1159 > 1024). Running this sequence through the model will result in indexing errors


86.66666666666667
87.17948717948718
87.6923076923077
88.2051282051282
88.71794871794872
89.23076923076924
89.74358974358975
90.25641025641026
90.76923076923077
91.28205128205128
91.7948717948718
92.3076923076923
92.82051282051282
93.33333333333333
93.84615384615384
94.35897435897435
94.87179487179486
95.38461538461539
95.8974358974359
96.41025641025641
96.92307692307692
97.43589743589743
97.94871794871794
98.46153846153847
98.97435897435898
99.48717948717949


In [ ]:
sum_data = pd.DataFrame(summary_list, columns = ['Statement', 'Answer', 'Label', 'Type'])

In [ ]:
sum_data.to_csv('train_data.csv')